# プロコン競技部門ベンチマーク

ライブラリをインストールする

In [ ]:
# Colab 用パッケージインストール
%pip install pandas matplotlib ipywidgets

problem.csvを読み込む

In [ ]:
# Colab のファイルアップローダー
from google.colab import files
import io

# アップロードダイアログを表示
uploaded = files.upload()

# アップロードされた problems_summary.csv / answer.csv を読み込む
problem_df = pd.read_csv(io.BytesIO(uploaded['problems_summary.csv']))
answers_df = pd.read_csv(io.BytesIO(uploaded['answer.csv']))

# actual_moves列の正規化とint変換（全角数字対策）
answers_df['actual_moves'] = (
    answers_df['actual_moves']
    .astype(str)
    .str.normalize('NFKC')
    .astype(int)
)

# 回答結果の読み込みと評価
このセクションでは、回答結果CSVを読み込み、各指標を算出し、総合スコアを計算、最後にグラフ化します。


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# 回答結果CSVの読み込み
answers_df = pd.read_csv('answers.csv')  # 適宜パスを調整

# 問題データ(problem_df)と結合
df = pd.merge(answers_df, problem_df, on='problem_id')

# 時間列を秒数に変換 (フォーマット "MM:SS" または "H:MM:SS" を想定)
def to_seconds(t):
    parts = list(map(int, t.split(':')))
    if len(parts) == 2:
        return parts[0] * 60 + parts[1]
    return parts[0] * 3600 + parts[1] * 60 + parts[2]
df['time_sec'] = df['time'].apply(to_seconds)

# 各指標の算出
total_time = df['time_sec'].sum()
avg_time = df['time_sec'].mean()
# 最適手数との差分
df['opt_gap'] = (df['actual_moves'] - df['optimal_moves']).abs()
# 理論最大値との差分 n^2
df['theory_gap'] = (df['actual_moves'] - df['n_squared']).abs()
# マンハッタン最小値・最大値との差分
df['man_min_gap'] = (df['actual_moves'] - df['manhattan_min']).abs()
df['man_max_gap'] = (df['actual_moves'] - df['manhattan_max']).abs()

# スコア重み設定
weights = {
    'w_total_time': 0.1,
    'w_avg_time': 0.1,
    'w_opt_gap': 0.25,
    'w_theory_gap': 0.15,
    'w_man_min': 0.2,
    'w_man_max': 0.2
}

# 正規化関数 (小さいほど良い指標を 1->0 にマッピング)
def score_inverse(series):
    return 1 - (series - series.min()) / (series.max() - series.min())

# 個別スコア計算
score_total_time = score_inverse(pd.Series([total_time]))
score_avg_time   = score_inverse(pd.Series([avg_time]))
score_opt_gap    = score_inverse(df['opt_gap']).mean()
score_theory_gap = score_inverse(df['theory_gap']).mean()
score_man_min    = score_inverse(df['man_min_gap']).mean()
score_man_max    = score_inverse(df['man_max_gap']).mean()

# 総合スコア
composite_score = (
    weights['w_total_time'] * score_total_time.iloc[0] +
    weights['w_avg_time']   * score_avg_time.iloc[0] +
    weights['w_opt_gap']    * score_opt_gap +
    weights['w_theory_gap'] * score_theory_gap +
    weights['w_man_min']    * score_man_min +
    weights['w_man_max']    * score_man_max
)
print(f"総合スコア: {composite_score:.4f}")

# 各問題ごとの総合スコア計算
# 問題ごとに同じ重みでスコア計算する場合の例
df_scores = pd.DataFrame({
    'problem_id': df['problem_id'],
    'score_opt_gap': score_inverse(df['opt_gap']),
    'score_theory_gap': score_inverse(df['theory_gap']),
    'score_man_min': score_inverse(df['man_min_gap']),
    'score_man_max': score_inverse(df['man_max_gap'])
})
df_scores['composite'] = (
    weights['w_opt_gap']    * df_scores['score_opt_gap'] +
    weights['w_theory_gap'] * df_scores['score_theory_gap'] +
    weights['w_man_min']    * df_scores['score_man_min'] +
    weights['w_man_max']    * df_scores['score_man_max']
)

# グラフ化
plt.figure()
df['time_sec'].hist(bins=20)
plt.title('解答時間分布')
plt.xlabel('解答時間 (秒)')
plt.ylabel('問題数')

plt.figure()
plt.plot(df_scores['problem_id'], df_scores['composite'], marker='o')
plt.title('問題ごとの総合スコア')
plt.xlabel('Problem ID')
plt.ylabel('Composite Score')

plt.show()
